In [ ]:
# ============================================================================
# CELL 0: ENVIRONMENT SETUP (Dev Only - Internet Required)
# ============================================================================
!pip install vllm==0.10.2 transformers datasets huggingface_hub --quiet

import json
import os
from pathlib import Path

# Create output directories
os.makedirs('/kaggle/working/test_dataset', exist_ok=True)
os.makedirs('/kaggle/working/utility_wheels', exist_ok=True)

In [ ]:
# Download bitsandbytes + accelerate wheels for offline use
!pip download bitsandbytes accelerate -d /kaggle/working/utility_wheels/ --no-deps
!ls -la /kaggle/working/utility_wheels/

In [ ]:
# ============================================================================
# CELL 1: CREATE TEST DATASET (50 AIMO-Caliber Problems)
# ============================================================================

TEST_PROBLEMS = [
    # === NUMBER THEORY (15 problems) ===
    {"id": "nt_001", "problem": "Find the remainder when 2^100 is divided by 7.", "answer": 2},
    {"id": "nt_002", "problem": "How many positive integers n ≤ 1000 are divisible by exactly one of 6 and 15?", "answer": 267},
    {"id": "nt_003", "problem": "Find the last three digits of 7^999.", "answer": 143},
    {"id": "nt_004", "problem": "What is the sum of all positive divisors of 360?", "answer": 1170},
    {"id": "nt_005", "problem": "Find the smallest positive integer n such that n! is divisible by 10^10.", "answer": 45},
    {"id": "nt_006", "problem": "How many integers from 1 to 1000 are coprime to 1000?", "answer": 400},
    {"id": "nt_007", "problem": "Find the remainder when 1! + 2! + 3! + ... + 100! is divided by 15.", "answer": 3},
    {"id": "nt_008", "problem": "What is the largest prime factor of 2^16 - 1?", "answer": 257},
    {"id": "nt_009", "problem": "Find the number of ordered pairs (a,b) of positive integers with a ≤ b and lcm(a,b) = 72.", "answer": 15},
    {"id": "nt_010", "problem": "What is the sum of all positive integers n < 100 such that n^2 + 1 is divisible by 17?", "answer": 392},
    {"id": "nt_011", "problem": "Find the remainder when 13^100 is divided by 1000.", "answer": 1},
    {"id": "nt_012", "problem": "How many positive integers n ≤ 500 satisfy gcd(n, 500) = 5?", "answer": 80},
    {"id": "nt_013", "problem": "Find the number of trailing zeros in 200!.", "answer": 49},
    {"id": "nt_014", "problem": "What is the smallest positive integer that has exactly 12 positive divisors?", "answer": 60},
    {"id": "nt_015", "problem": "Find the remainder when 3^3^3 is divided by 100.", "answer": 87},
    
    # === COMBINATORICS (15 problems) ===
    {"id": "comb_001", "problem": "How many 5-digit positive integers have digits that sum to 10?", "answer": 4995},
    {"id": "comb_002", "problem": "In how many ways can 8 people be seated at a round table if 2 specific people must not sit adjacent?", "answer": 3600},
    {"id": "comb_003", "problem": "How many subsets of {1,2,3,...,10} contain no two consecutive integers?", "answer": 144},
    {"id": "comb_004", "problem": "Find the number of ways to tile a 2×10 board with 1×2 dominoes.", "answer": 89},
    {"id": "comb_005", "problem": "How many lattice paths from (0,0) to (6,6) do not pass through (3,3)?", "answer": 1716},
    {"id": "comb_006", "problem": "In how many ways can 10 identical balls be distributed into 4 distinct boxes with each box containing at least one ball?", "answer": 84},
    {"id": "comb_007", "problem": "How many permutations of MISSISSIPPI have no two I's adjacent?", "answer": 5040},
    {"id": "comb_008", "problem": "Find the coefficient of x^5 in (1+x+x^2+x^3)^4.", "answer": 52},
    {"id": "comb_009", "problem": "How many 6-digit numbers have all digits nonzero and distinct, and are divisible by 5?", "answer": 8400},
    {"id": "comb_010", "problem": "In how many ways can we color the vertices of a square with 3 colors such that adjacent vertices have different colors?", "answer": 18},
    {"id": "comb_011", "problem": "How many binary strings of length 12 have exactly 4 ones and no two ones adjacent?", "answer": 126},
    {"id": "comb_012", "problem": "Find the number of surjective functions from a 5-element set to a 3-element set.", "answer": 150},
    {"id": "comb_013", "problem": "How many 4-element subsets of {1,2,...,12} have the property that no two elements differ by 1?", "answer": 126},
    {"id": "comb_014", "problem": "In how many ways can we arrange the letters of ARRANGEMENT so that all R's are together?", "answer": 151200},
    {"id": "comb_015", "problem": "How many positive integers less than 1000000 have digit sum equal to 7?", "answer": 462},
    
    # === ALGEBRA/POLYNOMIALS (10 problems) ===
    {"id": "alg_001", "problem": "Let r, s, t be the roots of x^3 - 6x^2 + 11x - 6 = 0. Find r^2 + s^2 + t^2.", "answer": 14},
    {"id": "alg_002", "problem": "Find the sum of all real roots of 8x^3 - 12x^2 - 6x + 9 = 0.", "answer": 1},
    {"id": "alg_003", "problem": "If f(x) = x^4 + x^3 + x^2 + x + 1, find the remainder when f(x^5) is divided by f(x).", "answer": 5},
    {"id": "alg_004", "problem": "Find the coefficient of x^3 in the expansion of (1+x)^5 + (1+x)^6 + (1+x)^7.", "answer": 85},
    {"id": "alg_005", "problem": "Let a and b be roots of x^2 - 5x + 3 = 0. Find a^4 + b^4.", "answer": 433},
    {"id": "alg_006", "problem": "Find the sum of all integer solutions to |x^2 - 10x + 21| = 4.", "answer": 20},
    {"id": "alg_007", "problem": "If x + 1/x = 3, find x^5 + 1/x^5.", "answer": 123},
    {"id": "alg_008", "problem": "Find the number of real solutions to x^4 - 4x^3 + 4x^2 + 1 = 0.", "answer": 0},
    {"id": "alg_009", "problem": "Let P(x) = x^3 + ax + b have roots r, s, t with r^2 + s^2 + t^2 = 2. Find a.", "answer": -1},
    {"id": "alg_010", "problem": "Find the product of all positive real solutions to x^(log_10(x)) = x^3 / 100.", "answer": 100},
    
    # === GEOMETRY (10 problems) ===
    {"id": "geo_001", "problem": "A triangle has sides 13, 14, 15. Find its area.", "answer": 84},
    {"id": "geo_002", "problem": "In a triangle with sides 5, 12, 13, find the length of the altitude to the longest side.", "answer": 4},
    {"id": "geo_003", "problem": "Find the number of integer-sided triangles with perimeter 15.", "answer": 7},
    {"id": "geo_004", "problem": "A regular hexagon has side length 6. Find its area.", "answer": 54},
    {"id": "geo_005", "problem": "In triangle ABC, AB=7, BC=8, CA=9. Find the length of the median from A to BC.", "answer": 7},
    {"id": "geo_006", "problem": "A circle of radius 5 has a chord of length 8. Find the distance from the center to the chord.", "answer": 3},
    {"id": "geo_007", "problem": "Find the area of the triangle with vertices (0,0), (4,0), (2,6).", "answer": 12},
    {"id": "geo_008", "problem": "Two circles of radii 3 and 5 have centers 6 apart. Find the length of their common chord.", "answer": 4},
    {"id": "geo_009", "problem": "In a right triangle with legs 6 and 8, find the radius of the inscribed circle.", "answer": 2},
    {"id": "geo_010", "problem": "A rectangle has diagonal 10 and perimeter 28. Find its area.", "answer": 48},
]

# Write as JSONL (Kaggle competition format)
with open('/kaggle/working/test_dataset/test.jsonl', 'w') as f:
    for p in TEST_PROBLEMS:
        # Competition format: id and problem only (answer for our validation)
        f.write(json.dumps({"id": p["id"], "problem": p["problem"]}) + '\n')

# Write answer key separately
with open('/kaggle/working/test_dataset/answers.json', 'w') as f:
    json.dump({p["id"]: p["answer"] for p in TEST_PROBLEMS}, f, indent=2)

print(f"✅ Created {len(TEST_PROBLEMS)} test problems")
print(f"📁 Location: /kaggle/working/test_dataset/")

In [ ]:
# ============================================================================
# CELL 2: DOWNLOAD UTILITY WHEELS FOR OFFLINE USE
# ============================================================================
# Download wheels that can be installed offline

!pip download vllm==0.10.2 -d /kaggle/working/utility_wheels/ --no-deps
!pip download xformers==0.0.32.post1 -d /kaggle/working/utility_wheels/ --no-deps  
!pip download ray==2.52.0 -d /kaggle/working/utility_wheels/ --no-deps

# List what we got
import os
wheels = os.listdir('/kaggle/working/utility_wheels/')
print(f"Downloaded {len(wheels)} wheel files")
for w in wheels:
    print(f"  - {w}")

In [ ]:
# ============================================================================
# CELL 3: CREATE DATASET METADATA
# ============================================================================

dataset_metadata = {
    "title": "AIMO3 Test Problems",
    "id": "your-username/aimo3-test-problems",  # CHANGE THIS
    "licenses": [{"name": "CC0-1.0"}]
}

with open('/kaggle/working/test_dataset/dataset-metadata.json', 'w') as f:
    json.dump(dataset_metadata, f, indent=2)

print("✅ Dataset ready for upload")
print("\n📋 MANUAL STEPS:")
print("1. Download /kaggle/working/test_dataset/ folder")
print("2. Go to kaggle.com/datasets/new")
print("3. Upload the folder contents")
print("4. Name it 'aimo3-test-problems'")

In [ ]:
!pip download bitsandbytes accelerate -d /kaggle/working/utility_wheels/ --no-deps
!ls /kaggle/working/utility_wheels/